In [2]:
!jt -t grade3 -fs 90 -tfs 15 -nfs 115 -cellw 80% -T
import ROOT as R
import array
import numpy as np
import pandas as pd
import pickle
import datetime
%jsroot on
c = R.TCanvas()
import plotly.express as px
import plotly.graph_objects as go

Welcome to JupyROOT 6.16/00


In [2]:
import log_loader_time

In [3]:
chain=R.TChain("tree")
chain.Add("/media/alb/Removibile/dati/351/Sub_RUN_ana_*")


990

In [4]:
chunck_size=chain.GetEntries()


In [5]:
red=log_loader_time.reader("/media/alb/Removibile/dati_raw/", "/media/alb/Removibile/dati/")


In [6]:
dict_time=red.elab_on_run_dict(351)

In [7]:
def get_number_of_trig_per_sub(num):
    
    tree=R.TChain("tree")
    tree.Add("/media/alb/Removibile/dati/{}/Sub_RUN_ana_*".format(num))

    num_trig_strart = {}
    num_trig_end = {}
    num_trig_per_sub = {}
    hit_noise_per_sub={}
    hit_charge_per_sub={}
    
    for entryNum in range (0,chunck_size):
        #parte trigger
        tree.GetEntry(entryNum)
        subRunNo=getattr(tree ,"subRunNo")
        count=getattr(tree ,"count")
        if subRunNo in num_trig_strart.keys():
            if count < num_trig_strart[subRunNo]:
                num_trig_strart[subRunNo]=count
            if count > num_trig_end[subRunNo]:
                num_trig_end[subRunNo]=count
        else:
            num_trig_strart[subRunNo]=count
            num_trig_end[subRunNo]=count

        #parte hit
        subRunNo=getattr(tree ,"subRunNo")
        charge_SH=getattr(tree ,"charge_SH")
        delta_coarse=getattr(tree ,"delta_coarse")
        l1ts_min_tcoarse=getattr(tree ,"l1ts_min_tcoarse")
        if delta_coarse==25 or delta_coarse==26:
            if (l1ts_min_tcoarse<1370 or l1ts_min_tcoarse>1430):
                if subRunNo in hit_noise_per_sub.keys():
                    hit_noise_per_sub[subRunNo]+=1
                else:
                    hit_noise_per_sub[subRunNo]=0
            elif (l1ts_min_tcoarse>1380 and l1ts_min_tcoarse<1420) and charge_SH>30:
                if subRunNo in hit_charge_per_sub.keys():
                    hit_charge_per_sub[subRunNo]+=1
                else:
                    hit_charge_per_sub[subRunNo]=0

    tot=0
    for key in num_trig_strart.keys():
        tot += num_trig_end[key]-num_trig_strart[key]
        num_trig_per_sub[key]=num_trig_end[key]-num_trig_strart[key]

    return tot,num_trig_per_sub,hit_noise_per_sub,hit_charge_per_sub

In [45]:
 tot,num_trig_per_sub,hit_noise_per_sub,hit_charge_per_sub=get_number_of_trig_per_sub(351)

KeyboardInterrupt: 

In [ ]:
data_4_pd={
    "sub_num":[],
    "charge_hits":[],
    "noise_hits":[],
    "time":[],
    "triggers":[],
    "error_charge":[],
    "error_noise":[]

}
for key in num_trig_per_sub:
    if  key in num_trig_per_sub.keys() and key in hit_charge_per_sub.keys() and key in hit_noise_per_sub.keys() and num_trig_per_sub[key]>50:
        data_4_pd["sub_num"].append(key)
        data_4_pd["charge_hits"].append(hit_charge_per_sub[key]/num_trig_per_sub[key])
        data_4_pd["noise_hits"].append(hit_noise_per_sub[key]/num_trig_per_sub[key])
        data_4_pd["time"].append(dict_time["{}".format(key)])
        data_4_pd["triggers"].append(num_trig_per_sub[key])
        data_4_pd["error_charge"].append(  (((hit_charge_per_sub[key]  )**(1/2)/ num_trig_per_sub[key])**2+((num_trig_per_sub[key]**(1/2))/num_trig_per_sub[key]**2*hit_charge_per_sub[key])**2)**(1/2) )
        data_4_pd["error_noise"].append(( ((hit_noise_per_sub[key]  )**(1/2)/ num_trig_per_sub[key]   )**2+((num_trig_per_sub[key]**(1/2))/num_trig_per_sub[key]**2*hit_noise_per_sub[key])**2)**1/2  )


    else:
        print ("Can't save for suburn {}".format(key))

In [32]:
df_sub=pd.DataFrame(data_4_pd)
df_sub['time'] = pd.to_datetime(df_sub['time'],unit='s')


In [33]:
df_sub=df_sub.set_index("time")

In [34]:
df_sub["hour"]=df_sub.index.hour
df_sub["day"]=df_sub.index.date

In [35]:
df_sub.to_pickle("vs_time_analysys_save/351")

In [76]:
fig = px.scatter(df_sub,y="charge_hits",error_y="error_charge",hover_data=["triggers"])

In [77]:
fig.show()

In [64]:
fig = px.box(df_sub, x="hour", y="charge_hits",hover_data=["triggers"])

In [65]:
fig.show()

In [75]:
fig = px.box(df_sub, x="day", y="charge_hits",hover_name="sub_num",hover_data=["triggers"])
fig.show()

In [67]:
fig = px.scatter(df_sub,y="noise_hits",error_y="error_noise")
fig.show()

In [68]:
fig = px.box(df_sub, x="hour", y="noise_hits")
fig.show()

In [63]:
fig = px.box(df_sub, x="day", y="noise_hits")
fig.show()

In [44]:
fig = px.scatter(df_sub, y="triggers")
fig.show()

In [5]:
df_sub=pd.read_pickle("/home/alb/Desktop/elaborazioni_e_dati/analisi_run/vs_time_analysys_save/351")
df_sub

,sub_num,charge_hits,noise_hits,triggers,error_charge,error_noise,hour,day
time,,,,,,,,
2020-01-26 03:00:24,10,6.969799,43.312081,298,0.431743,3.220215,3,2020-01-26
2020-01-26 18:08:19,100,6.732000,36.744000,250,0.456297,2.773731,18,2020-01-26
2020-02-01 23:51:07,1000,7.599278,41.212996,277,0.485710,3.140296,23,2020-02-01
2020-02-02 00:01:58,1001,6.848592,45.334507,284,0.435048,3.698155,0,2020-02-02
2020-02-02 00:12:52,1002,6.874539,43.985240,271,0.446940,3.650713,0,2020-02-02
2020-02-02 00:23:49,1003,6.386525,46.524823,282,0.409004,3.920362,0,2020-02-02
2020-02-02 00:34:41,1004,7.211538,48.484615,260,0.477243,4.613928,0,2020-02-02
2020-02-02 00:45:34,1005,6.567839,44.919598,199,0.499771,5.182638,0,2020-02-02
2020-02-02 00:53:33,1006,7.569620,51.687764,237,0.523171,5.745385,0,2020-02-02


In [8]:
df_sub["sig/noise"]=df_sub.charge_hits/df_sub.noise_hits

In [9]:
df_sub

,sub_num,charge_hits,noise_hits,triggers,error_charge,error_noise,hour,day,sig/noise
time,,,,,,,,,
2020-01-26 03:00:24,10,6.969799,43.312081,298,0.431743,3.220215,3,2020-01-26,0.160920
2020-01-26 18:08:19,100,6.732000,36.744000,250,0.456297,2.773731,18,2020-01-26,0.183214
2020-02-01 23:51:07,1000,7.599278,41.212996,277,0.485710,3.140296,23,2020-02-01,0.184390
2020-02-02 00:01:58,1001,6.848592,45.334507,284,0.435048,3.698155,0,2020-02-02,0.151068
2020-02-02 00:12:52,1002,6.874539,43.985240,271,0.446940,3.650713,0,2020-02-02,0.156292
2020-02-02 00:23:49,1003,6.386525,46.524823,282,0.409004,3.920362,0,2020-02-02,0.137271
2020-02-02 00:34:41,1004,7.211538,48.484615,260,0.477243,4.613928,0,2020-02-02,0.148739
2020-02-02 00:45:34,1005,6.567839,44.919598,199,0.499771,5.182638,0,2020-02-02,0.146213
2020-02-02 00:53:33,1006,7.569620,51.687764,237,0.523171,5.745385,0,2020-02-02,0.146449


In [15]:
fig = px.scatter(df_sub,x="hour",y="sig/noise")
fig.show()